####Loading the Data from Kaggle

In [38]:
import pandas as pd
import pickle
import numpy as np
import tensorflow as tf

In [39]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tobiasbueck/multilingual-customer-support-tickets")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'multilingual-customer-support-tickets' dataset.
Path to dataset files: /kaggle/input/multilingual-customer-support-tickets


In [40]:
import os

path_to_dataset = "/root/.cache/kagglehub/datasets/tobiasbueck/multilingual-customer-support-tickets/versions/14"

os.listdir(path_to_dataset)

['dataset-tickets-german_normalized.csv',
 'dataset-tickets-german_normalized_50_5_2.csv',
 'aa_dataset-tickets-multi-lang-5-2-50-version.csv',
 'dataset-tickets-multi-lang-4-20k.csv',
 'dataset-tickets-multi-lang3-4k.csv']

In [41]:
BASE_DIR = "/root/.cache/kagglehub/datasets/tobiasbueck/multilingual-customer-support-tickets/versions/14/dataset-tickets-multi-lang-4-20k.csv"

df = pd.read_csv(BASE_DIR)

print(df.head(20))

                                              subject  \
0   Unvorhergesehener Absturz der Datenanalyse-Pla...   
1                            Customer Support Inquiry   
2                       Data Analytics for Investment   
3                  Krankenhaus-Dienstleistung-Problem   
4                                            Security   
5   Concerns About Securing Medical Data on 2-in-1...   
6   Ratung für Sicherung medizinischer Daten in Hu...   
7                            Problem with Integration   
8                                  Assistance Request   
9                                     Support Request   
10  Issue with Data Analytics Platform - Insuffici...   
11  Reported Issue with Project Sync Resulting in ...   
12                     Strategies for Brand Expansion   
13             Issue with Website Analytics Dashboard   
14  Urgent: Data Breach Identified in Hospital Net...   
15       Request for Assistance with Data Integration   
16  Request for Updating Integr

####Creating the data format for the tokenization

In [42]:
df["subject"] = df["subject"].fillna("")
df["body"] = df["body"].fillna("")

df["Final_Text"] = df["subject"] + " " + df["body"]

In [43]:
df = df.dropna(subset=["queue"])

df = df[df["Final_Text"].str.strip() != ""]

In [44]:
print("Subject: \n", df["subject"], "\n")
print("Body \n", df["body"], "\n")
print("Final Text \n", df["Final_Text"], "\n")

Subject: 
 0        Unvorhergesehener Absturz der Datenanalyse-Pla...
1                                 Customer Support Inquiry
2                            Data Analytics for Investment
3                       Krankenhaus-Dienstleistung-Problem
4                                                 Security
                               ...                        
19995       Assistance Needed for IFTTT Docker Integration
19996          Bitten um Unterstützung bei der Integration
19997                                                     
19998              Hilfe bei digitalen Strategie-Problemen
19999    Optimierung Ihrer Datenanalyse-Plattform erlei...
Name: subject, Length: 20000, dtype: object 

Body 
 0        Die Datenanalyse-Plattform brach unerwartet ab...
1        Seeking information on digital strategies that...
2        I am contacting you to request information on ...
3        Ein Medien-Daten-Sperrverhalten trat aufgrund ...
4        Dear Customer Support, I am reaching out t

####Tokenizer

In [45]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_words = 15000
max_len = 200

tokenizer = Tokenizer(num_words=max_words, lower=True)
tokenizer.fit_on_texts(df["Final_Text"])

####Padding

In [46]:
X = pad_sequences(tokenizer.texts_to_sequences(df["Final_Text"]), maxlen=max_len)

In [47]:
X

array([[    0,     0,     0, ...,   223,    11,   346],
       [    0,     0,     0, ...,    87,     5,   127],
       [    0,     0,     0, ...,     1,  1860,  2057],
       ...,
       [    0,     0,     0, ...,     6,    12,    35],
       [    0,     0,     0, ...,    16,    11,   252],
       [    0,     0,     0, ...,    22,  1020, 11081]], dtype=int32)

####Lable encoding

In [48]:
dummy_ys = pd.get_dummies(df["queue"])
queue_names = dummy_ys.columns.tolist()
y = dummy_ys.values

print(y, "\n")
print(dummy_ys.head())

[[False False  True ... False False False]
 [False  True False ... False False False]
 [False  True False ... False False False]
 ...
 [ True False False ... False False False]
 [False False False ... False False False]
 [False False False ...  True False False]] 

   Billing and Payments  Customer Service  General Inquiry  Human Resources  \
0                 False             False             True            False   
1                 False              True            False            False   
2                 False              True            False            False   
3                 False              True            False            False   
4                 False              True            False            False   

   IT Support  Product Support  Returns and Exchanges  Sales and Pre-Sales  \
0       False            False                  False                False   
1       False            False                  False                False   
2       False            

In [49]:
#Train test splitting
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [50]:
print("X_train shape: ", X_train.shape)
print("X_val shape: ", X_val.shape)
print("y_train shape: ", y_train.shape)
print("y_val shape: ", y_val.shape)

X_train shape:  (16000, 200)
X_val shape:  (4000, 200)
y_train shape:  (16000, 10)
y_val shape:  (4000, 10)


In [51]:
print("X_train: \n", X_train, "\n")
print("y_train: \n", y_train, "\n")

X_train: 
 [[  0   0   0 ... 264 359 127]
 [  0   0   0 ... 181  11 856]
 [  0   0   0 ...  33  68 304]
 ...
 [  0   0   0 ...  18  14 376]
 [  0   0   0 ...  18  15 101]
 [  0   0   0 ... 540  39 256]] 

y_train: 
 [[False False False ... False  True False]
 [False False False ... False False  True]
 [ True False False ... False False False]
 ...
 [ True False False ... False False False]
 [False False False ... False False False]
 [ True False False ... False False False]] 



In [52]:
print(len(queue_names), "have been identified", "\n")
print("The queue names are: ", queue_names)

10 have been identified 

The queue names are:  ['Billing and Payments', 'Customer Service', 'General Inquiry', 'Human Resources', 'IT Support', 'Product Support', 'Returns and Exchanges', 'Sales and Pre-Sales', 'Service Outages and Maintenance', 'Technical Support']


####Model

In [53]:
model = tf.keras.Sequential([

    tf.keras.layers.Input(shape=(max_len,)),

    tf.keras.layers.Embedding(max_words, 128),
    tf.keras.layers.SpatialDropout1D(0.3),

    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64)
    ),

    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.4),

    tf.keras.layers.Dense(len(queue_names), activation="softmax")
])

In [54]:
loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss=loss,
    metrics=["accuracy"]
)

In [55]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ (None, 200, 128)       │     1,920,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_5             │ (None, 200, 128)       │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_5 (Bidirectional) │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,027,722 (7.74 MB)

 Trainable params: 2,027,722 (7.74 MB)

 Non-trainable params: 0 (0.00 B)

####Training the model

In [56]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=2,
    restore_best_weights=True
)

In [57]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=15, batch_size=32, callbacks=[early_stop])

Epoch 1/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 134s 260ms/step - accuracy: 0.2256 - loss: 2.1676 - val_accuracy: 0.2948 - val_loss: 2.0206
Epoch 2/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 138s 252ms/step - accuracy: 0.2895 - loss: 2.0406 - val_accuracy: 0.2945 - val_loss: 1.9732
Epoch 3/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 129s 259ms/step - accuracy: 0.3005 - loss: 1.9933 - val_accuracy: 0.3532 - val_loss: 1.9079
Epoch 4/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 142s 258ms/step - accuracy: 0.3616 - loss: 1.9067 - val_accuracy: 0.3655 - val_loss: 1.8515
Epoch 5/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 127s 254ms/step - accuracy: 0.3817 - loss: 1.8467 - val_accuracy: 0.3702 - val_loss: 1.8444
Epoch 6/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 145s 261ms/step - accuracy: 0.4007 - loss: 1.8086 - val_accuracy: 0.3798 - val_loss: 1.8464
Epoch 7/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 137s 252ms/step - accuracy: 0.4225 - loss: 1.7753 - val_accuracy: 0.3825 - val_loss: 1.8498
